In [12]:
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import mnist
from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam

In [5]:
(X_train,_),(_,_) = mnist.load_data()

11493376/11490434 [==============================] - 17s 2us/step


In [26]:
print(X_train.shape)

(60000, 784)


In [27]:
# normalization

X_train = (X_train.astype('float32')-127.5)/127.5
print(np.min(X_train))
print(np.max(X_train))

-1.0078431
-0.99215686


In [28]:
X_train = X_train.reshape(-1,28*28)
print(X_train.shape)

(60000, 784)


In [29]:
EPOCHS = 60
BATCH_SIZE = 256
NO_OF_BATCHES = int(X_train.shape[0]/BATCH_SIZE)
HALF_BATCH_SIZE = 128
NOISE_DIM = 100

adam = Adam(lr=2e-4,beta_1=0.5)

In [30]:
# GENERATOR

generator = Sequential()
generator.add(Dense(256,input_shape=(NOISE_DIM,)))
generator.add(LeakyReLU(0.2))
generator.add(Dense(512))
generator.add(LeakyReLU(0.2))
generator.add(Dense(1024))
generator.add(LeakyReLU(0.2))
generator.add(Dense(784,activation='tanh'))

#compile
generator.compile(loss='binary_crossentropy', optimizer=adam)

generator.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_20 (LeakyReLU)   (None, 1024)              0         
_________________________________________________________________
dense_28 (Dense)             (None, 784)               803600    
Total para

In [34]:
discriminator = Sequential()

discriminator.add(Dense(512,input_shape=(784,)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dense(1,activation='sigmoid'))

discriminator.compile(loss = 'binary_crossentropy',optimizer=adam)

discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Combined Model (generator + Discrimintor)->Function API

discriminator.trainable = False
gan_input = Input(shape=(NOISE_DIM,))
generator_output = generator(gan_input)
gan_output = discriminator(generator_output)

model = Model(gan_input,gan_output)
model.compile(loss='binary_crossentropy', optimizer=adam)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 100)               0         
_________________________________________________________________
sequential_9 (Sequential)    (None, 784)               1486352   
_________________________________________________________________
sequential_11 (Sequential)   (None, 1)                 533505    
Total params: 2,019,857
Trainable params: 1,486,352
Non-trainable params: 533,505
_________________________________________________________________


In [38]:
# training Loop

for epoch in range(EPOCHS):
    epoch_d_loss = 0.0
    epoch_g_loss = 0.0
    
    #mini batch
    for step in range(NO_OF_BATCHES):
        idx = np.random.randint(0,X_train.shape[0],HALF_BATCH_SIZE)
        real_imgs = X_train[idx]
        
        # generate fake images asaumong generator is frozen
        noise = np.random.normal(0,1,size=(HALF_BATCH_SIZE,NOISE_DIM))
        fake_imgs = generator.predict(noise)
        
        # labels
        
        real_y = np.ones((HALF_BATCH_SIZE,1))*0.9
        fake_y = np.zeros((HALF_BATCH_SIZE,1))
        # train on real and fake imgs
        d_real_loss = discriminator.train_on_batch(real_imgs,real_y)
        d_fake_loss = discriminator.train_on_batch(fake_imgs,fake_y)
        
        d_loss = 0.5*d_real_loss + 0.5*d_fake_loss
        epoch_d_loss += d_loss
        
        # train Generator
        noise = np.random.normal(0,1,size=(BATCH_SIZE,NOISE_DIM))
        real_y = np.ones((BATCH_SIZE,1))
        g_loss = model.train_on_batch(noise,real_y)
        epoch_g_loss += g_loss
        
    print("Epoch %d D loss %.4f G loss %0.4f "%((epoch+1),epoch_d_loss,epoch_g_loss))
    

Epoch 1 D loss 157.7135 G loss 217.6108 


KeyboardInterrupt: 